# Learnplatform Covid19 Impact on Digital Learning

## This Notebook is deivided into Two main parts

### 1. Data Pre-processing
### 2. Exploratory Data Analysis (EDA)

## Data Reading and Pre-processing

### Importing Required packages and Libraries

In [2]:
# Importing necessary packages

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

import warnings
warnings.filterwarnings("ignore")


### Path to different Data files, scripts files and files 

In [3]:
districts_data_path = "../data/districts_info.csv"
products_data_path = "../data/products_info.csv"
engagements_data_path = "../data/engagement_data/"

### Loading datasets

In [5]:
districts_data = pd.read_csv(districts_data_path)
products_data = pd.read_csv(products_data_path)

products_data.info()
districts_data.info()
print(f" Shape of Products_data: {products_data.shape}, and that of District_data is: {districts_data.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   LP ID                       372 non-null    int64 
 1   URL                         372 non-null    object
 2   Product Name                372 non-null    object
 3   Provider/Company Name       371 non-null    object
 4   Sector(s)                   352 non-null    object
 5   Primary Essential Function  352 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   district_id               233 non-null    int64 
 1   state                     176 non-null    object
 2   locale                    176 non-null    object
 3   pct_black/hispanic

### Showing sample data

In [7]:
products_data.sample(10)

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
371,36254,http://www.truenorthlogic.com/,True North Logic,NaN,NaN,NaN
349,70878,https://musiclab.chromeexperiments.com/,CHROME MUSIC LAB,Google LLC,PreK-12; Higher Ed,LC - Content Creation & Curation
216,70167,https://www.microsoft.com/en-us/download/detai...,Microsoft XNA Game Studio,Microsoft,PreK-12; Higher Ed; Corporate,LC - Content Creation & Curation
329,54110,https://www.pandora.com/,Pandora,"Pandora Media, LLC",PreK-12; Higher Ed,"LC - Sites, Resources & Reference - Streaming ..."
167,78063,https://www.classlink.com/,ClassLink,ClassLink,PreK-12; Higher Ed,SDO - School Management Software - SSO
82,16468,http://www.archives.gov/,National Archives,United States National Archives,PreK-12; Higher Ed,"LC - Sites, Resources & Reference - Digital Co..."
362,70706,https://canvas.apps.chrome/,Canvas for Chrome,"Canvas Talent, Inc.",PreK-12; Higher Ed; Corporate,LC - Content Creation & Curation
156,58235,http://ed.ted.com/,TedEd,TED Conferences,PreK-12; Higher Ed,"LC - Sites, Resources & Reference"
280,16401,https://www.pbs.org/,PBS,PBS,PreK-12; Higher Ed; Corporate,"LC - Sites, Resources & Reference"
337,37948,https://www.docusign.com/,DocuSign,DocuSign Inc,PreK-12; Higher Ed; Corporate,SDO - Other


In [71]:
districts_data.sample(4)

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
22,2167,Washington,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
161,4314,New Hampshire,Rural,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[",NaN
27,7785,Connecticut,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[",NaN
88,8937,Illinois,Rural,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[12000, 14000["


### Calculating the Missing Values, NA

In [65]:
# Functions to Calculate Missing Values and droping columns

def drop_columns(self, df, column_list):
        df_new = df.drop(column_list, axis=1)

        return df_new

def total_percent_missing_data(df):

        # Calculate total number of cells in dataframe
        totalCells = np.product(df.shape)

        # Count number of missing values per column
        missingCount = df.isnull().sum()

        # Calculate total number of missing values
        totalMissing = missingCount.sum()

        # Calculate percentage of missing values
        return round(((totalMissing/totalCells) * 100), 2)


def missing_data_per_column(df):
        item_list = []
        row_list = []
        new_columns=['Column', 'No. of Missing Values', '% Missing Values per column']
        total_no_data_per_column = df.shape[0]-1
        i=0
        for item in df.columns:
            no_missing_values = df[item].isna().sum()
            percentage = str(round(((no_missing_values/total_no_data_per_column) * 100), 2))+" %"
            row_list.append(item)
            row_list.append(no_missing_values)
            row_list.append(percentage)
            item_list.append(row_list)
            row_list = []

        df_data = pd.DataFrame(item_list, columns = new_columns)
        return df_data


missing_values = total_percent_missing_data(districts_data)
missing_df = missing_data_per_column(districts_data)

print(f" Summary of Missing Values in districts_data : {missing_values} %")
print("Missing values per column in districts_data")
missing_df

 Summary of Missing Values in districts_data : 27.1 %
Missing values per column in districts_data


,Column,No. of Missing Values,% Missing Values per column
0,district_id,0,0.0 %
1,state,57,24.57 %
2,locale,57,24.57 %
3,pct_black/hispanic,57,24.57 %
4,pct_free/reduced,85,36.64 %
5,county_connections_ratio,71,30.6 %
6,pp_total_raw,115,49.57 %


In [67]:
# Checking for Pattern in Missing Values
nan_rows = districts_data[districts_data['state'].isna() & districts_data['locale'].isna()]
nan_rows1 = districts_data[districts_data['state'].isna()].count().sum()
nan_rows2 = districts_data[districts_data['state'].isna() & districts_data['locale'].isna() & districts_data['pct_black/hispanic'].isna() & districts_data['pct_free/reduced'].isna() & districts_data['county_connections_ratio'].isna() & districts_data['pp_total_raw'].isna()].count().sum()
print(f" The number of columns with Missing 'state' values and Missinng all columns except 'district_id' are:  {nan_rows1}, {nan_rows2}")
nan_rows.sample(10)

 The number of columns with Missing 'state' values and Missinng all columns except 'district_id' are:  57, 57


,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
174,8796,NaN,NaN,NaN,NaN,NaN,NaN
164,1039,NaN,NaN,NaN,NaN,NaN,NaN
162,4937,NaN,NaN,NaN,NaN,NaN,NaN
23,9120,NaN,NaN,NaN,NaN,NaN,NaN
222,9463,NaN,NaN,NaN,NaN,NaN,NaN
199,4165,NaN,NaN,NaN,NaN,NaN,NaN
215,3390,NaN,NaN,NaN,NaN,NaN,NaN
106,3371,NaN,NaN,NaN,NaN,NaN,NaN
216,5404,NaN,NaN,NaN,NaN,NaN,NaN
3,3188,NaN,NaN,NaN,NaN,NaN,NaN


### It can be concluded, that all data that have missing state, have also missing locale, pct_black/hispanic, pct_free/reduced, county_connections_ratio, pp_total_raw

### Droping rows having null values

In [68]:
# Droping all rows with null values in state, locale, etc
districts_data = districts_data[districts_data.state.notna()].reset_index(drop=True)
districts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   district_id               176 non-null    int64 
 1   state                     176 non-null    object
 2   locale                    176 non-null    object
 3   pct_black/hispanic        176 non-null    object
 4   pct_free/reduced          148 non-null    object
 5   county_connections_ratio  162 non-null    object
 6   pp_total_raw              118 non-null    object
dtypes: int64(1), object(6)
memory usage: 9.8+ KB


In [69]:
missing_values = total_percent_missing_data(products_data)
missing_df = missing_data_per_column(products_data)

print(f" Summary of Missing Values in products_data : {missing_values} %")
print("Missing values per column in products_data")
missing_df

 Summary of Missing Values in products_data : 1.84 %
Missing values per column in products_data


,Column,No. of Missing Values,% Missing Values per column
0,LP ID,0,0.0 %
1,URL,0,0.0 %
2,Product Name,0,0.0 %
3,Provider/Company Name,1,0.27 %
4,Sector(s),20,5.39 %
5,Primary Essential Function,20,5.39 %


### Filling Missing Values, Using Median values

In [21]:
districts_data['pp_total_raw']
products_data['Sector(s)'].unique()

array(['PreK-12', 'PreK-12; Higher Ed', 'PreK-12; Higher Ed; Corporate',
       nan, 'Corporate', 'Higher Ed; Corporate'], dtype=object)